In [13]:
from thinc.layers.sparselinear import forward
!pip install diffusers transformers accelerate
!pip install -U xformers torch torchvision torchaudio

In [14]:
!pip install -U --pre triton

ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton


In [15]:
import os
import requests
from dotenv import load_dotenv
from huggingface_hub import login

In [16]:
load_dotenv(override=True)
hf_token = os.getenv('HF_TOKEN')
login(token=hf_token)
os.environ["XFORMERS_DISABLE_TRITON"] = "1"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Will only work on powerful GPUs

In [17]:
# import torch
# from diffusers import FluxPipeline
#
# FLUX_MODEL ="black-forest-labs/FLUX.1-schnell"
#
# pipe = FluxPipeline.from_pretrained(FLUX_MODEL, torch_dtype=torch.bfloat16).to("cuda")
#
# generator = torch.Generator(device="cude").manual_seed(0)
# prompt = "A futiristic class full of students learning AI coding in the surrea; style of Salvador Dali"
# image = pipe(
#     prompt,
#     guidance_scale=0.0,
#     num_inference_steps=3,
#     max_sequence_length=256,
#     generator=generator,
# ).images[0]
#
# image.save("surreal.png")

In [19]:
# Import required libraries
from transformers import pipeline  # Hugging Face's transformers library for pre-trained models
from datasets import load_dataset  # For loading voice embedding datasets
import soundfile as sf  # For saving audio files
import torch  # PyTorch for tensor operations

In [35]:
# Define constants for model and voice embeddings
TTS_MODEL = "microsoft/speecht5_tts"  # Microsoft's SpeechT5 text-to-speech model
EMBEDDING_DATASET = "Matthijs/cmu-arctic-xvectors"  # Dataset containing speaker voice embeddings

# Initialize the text-to-speech pipeline with CUDA acceleration
synthesiser = pipeline(
    task="text-to-speech",
    model=TTS_MODEL,
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

# Load the voice embeddings dataset and select a specific speaker
embeddings_dataset = load_dataset(EMBEDDING_DATASET)
print(embeddings_dataset.keys())  # Output: ['validation']
speaker_embedding = torch.tensor(embeddings_dataset['validation'][7306]["xvector"]).unsqueeze(0)
# unsqueeze(0) adds a batch dimension required by the model

# Generate speech with custom parameters
speech = synthesiser(
    text_inputs="Hi to an artificial intelligence engineer on the way to mastery!",  # Input text
    forward_params={
        "speaker_embeddings": speaker_embedding  # Inject specific speaker characteristics
    }
)

# Save the generated audio to file
sf.write(
    file="speech.wav",
    data=speech["audio"],  # Audio data array
    samplerate=speech["sampling_rate"]  # Preserve original sample rate (usually 16kHz)
)

Device set to use cuda


dict_keys(['validation'])


In [36]:
from IPython.display import Audio

# Play the generated audio
Audio("speech.wav")

## low resource intensive image generation model

In [38]:
import torch
import os
from diffusers import StableDiffusionPipeline
os.environ["XFORMERS_DISABLE_TRITON"] = "1"
model_id = "OFA-Sys/small-stable-diffusion-v0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.enable_model_cpu_ofload()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

diffusion_pytorch_model.bin:  62%|######2   | 1.45G/2.32G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/93/9f/939fb9a47a4b704e7a302486326bc3223b829745df0ec0c87434da20b2e5bd2a/2cb22aebb48a8ba4f44bde337f5a0dc6fc251dd889829da2a54bc9e6e94fef41?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.bin%3B+filename%3D%22diffusion_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1743008972&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzAwODk3Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy85My85Zi85MzlmYjlhNDdhNGI3MDRlN2EzMDI0ODYzMjZiYzMyMjNiODI5NzQ1ZGYwZWMwYzg3NDM0ZGEyMGIyZTViZDJhLzJjYjIyYWViYjQ4YThiYTRmNDRiZGUzMzdmNWEwZGM2ZmMyNTFkZDg4OTgyOWRhMmE1NGJjOWU2ZTk0ZmVmNDE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=mQuYj-30Wbg%7EDwBR-pdpZS6fbTNu6vTvhOFa-PFN8tRz3f715spgvnFpJ%7EbPmwjrUhQWCbwm6GKz40nfrJa8lEccXfPqyKmDt217OHuacVj%7EylUnjG9fqrDgotXDoKkv0bCEHgO23A3PEHvdvKkreU6OiCeVfn8%

diffusion_pytorch_model.bin:  71%|#######   | 1.64G/2.32G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\ruhit\.cache\huggingface\hub\models--OFA-Sys--small-stable-diffusion-v0\snapshots\38e10e5e71e8fbf717a47a81e7543cd01c1a8140\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\ruhit\.cache\huggingface\hub\models--OFA-Sys--small-stable-diffusion-v0\snapshots\38e10e5e71e8fbf717a47a81e7543cd01c1a8140\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\ruhit\.cache\huggingface\hub\models--OFA-Sys--small-stable-diffusion-v0\snapshots\38e10e5e71e8fbf717a47a81e7543cd01c1a8140\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\ruhit\.cache\huggingface\hub\models--OFA-Sys--small-stable-diffusion-v0\snapshots\38e10e5e71e8fbf717a47a81e7543cd01c1a8140\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'predict_epsilon': 

AttributeError: 'StableDiffusionPipeline' object has no attribute 'enable_model_cpu_ofload'